# Clustering Crypto

In [146]:
# Initial imports
import pandas as pd
import hvplot.pandas
from path import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


### Deliverable 1: Preprocessing the Data for PCA

In [147]:
# Load the crypto_data.csv dataset.
file_path = "./Resources/crypto_data.csv"
crypto_df = pd.read_csv(file_path, index_col=0)
crypto_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [148]:
crypto_df.dtypes

CoinName            object
Algorithm           object
IsTrading             bool
ProofType           object
TotalCoinsMined    float64
TotalCoinSupply     object
dtype: object

In [149]:
# Keep all the cryptocurrencies that are being traded.
crypto_df = crypto_df[crypto_df.IsTrading]
crypto_df.head(5)

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [150]:
crypto_df.shape

(1144, 6)

In [151]:
# Keep all the cryptocurrencies that have a working algorithm.
crypto_df= crypto_df.dropna(subset=["Algorithm"]) 
crypto_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [152]:
crypto_df.shape

(1144, 6)

In [153]:
# Remove the "IsTrading" column. 
crypto_df.drop("IsTrading", axis=1, inplace=True)
crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
365,365Coin,X11,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,PoW,NaN,611000
808,808,SHA-256,PoW/PoS,0.000000e+00,0


In [154]:
crypto_df.shape

(1144, 5)

In [155]:
# Remove rows that have at least 1 null value.
crypto_df = crypto_df.dropna(how='any')
crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000


In [156]:
crypto_df.shape

(685, 5)

In [157]:
# Keep the rows where coins are mined.
crypto_df.drop(crypto_df[crypto_df.TotalCoinsMined <= 0].index, inplace=True)
crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0


In [158]:
crypto_df.shape

(532, 5)

In [159]:
# Create a new DataFrame that holds only the cryptocurrencies names.
coins_name = pd.DataFrame(crypto_df["CoinName"], index=crypto_df.index)
coins_name.head(5)

,CoinName
42,42 Coin
404,404Coin
1337,EliteCoin
BTC,Bitcoin
ETH,Ethereum


In [160]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
crypto_df.drop("CoinName", axis=1, inplace=True)
crypto_df.head(5)

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0


In [161]:
crypto_df.shape

(532, 4)

In [162]:
# Use get_dummies() to create variables for text features.
X = pd.get_dummies(crypto_df)
X.head()

,TotalCoinsMined,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,Algorithm_Cloverhash,...,TotalCoinSupply_91388946,TotalCoinSupply_92000000000,TotalCoinSupply_9354000,TotalCoinSupply_9507271,TotalCoinSupply_9736000,TotalCoinSupply_98000000,TotalCoinSupply_98100000000,TotalCoinSupply_990000000000,TotalCoinSupply_999481516,TotalCoinSupply_9999999
42,4.199995e+01,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
404,1.055185e+09,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [163]:
# Standardize the data with StandardScaler().
X_scaled = StandardScaler().fit_transform(X)
print(X_scaled[0:5])

[[-0.11710817 -0.0433963  -0.0433963  ... -0.0433963  -0.0433963
  -0.0433963 ]
 [-0.09396955 -0.0433963  -0.0433963  ... -0.0433963  -0.0433963
  -0.0433963 ]
 [ 0.52494561 -0.0433963  -0.0433963  ... -0.0433963  -0.0433963
  -0.0433963 ]
 [-0.11671506 -0.0433963  -0.0433963  ... -0.0433963  -0.0433963
  -0.0433963 ]
 [-0.11474682 -0.0433963  -0.0433963  ... -0.0433963  -0.0433963
  -0.0433963 ]]


### Deliverable 2: Reducing Data Dimensions Using PCA

In [164]:
# Using PCA to reduce dimension to three principal components.
pca = PCA(n_components=3)
X_pca = pca.fit_transform(X_scaled)
X_pca


array([[-0.30585866, -0.1231534 , -0.01250849],
       [-0.2975962 , -0.12705206, -0.00766391],
       [ 0.3038023 , -0.12884349, -0.13854279],
       ...,
       [-0.0768311 , -0.34147626, -0.1604541 ],
       [-0.29868764, -0.14229246, -0.04766424],
       [-0.19586088, -0.09451104, -0.00861686]])

In [165]:
# Create a DataFrame with the three principal components.
pcs_df=pd.DataFrame(data=X_pca, columns=["PC 1", "PC 2", "PC 3"], index=crypto_df.index)
pcs_df.head()

,PC 1,PC 2,PC 3
42,-0.305859,-0.123153,-0.012508
404,-0.297596,-0.127052,-0.007664
1337,0.303802,-0.128843,-0.138543
BTC,-0.301184,-0.176797,-0.016435
ETH,-0.089452,-0.238902,-0.072992


In [166]:
pcs_df.shape

(532, 3)

### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [167]:
# Create an elbow curve to find the best value for K.
# Find the best value for K
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of K values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(pcs_df)
    inertia.append(km.inertia_)

# Create the elbow curve
elbow_data = {"k": k, "inertia": inertia}
elbow_df = pd.DataFrame(elbow_data)
elbow_df.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Curve")


C:\Users\girar\anaconda3\envs\PythonData\lib\site-packages\sklearn\cluster\_kmeans.py:882: UserWarning:

KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.



:Curve   [k]   (inertia)

Running K-Means with `k=4`

In [168]:
# Initialize the K-Means model.
model = KMeans(n_clusters=4, random_state=0)


# Fit the model
model.fit(pcs_df)

# Predict clusters
predictions = model.predict(pcs_df)
predictions

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [169]:
# Now let's add the new predicted class as a columns
pcs_df["class"] = model.labels_
pcs_df.head()

,PC 1,PC 2,PC 3,class
42,-0.305859,-0.123153,-0.012508,0
404,-0.297596,-0.127052,-0.007664,0
1337,0.303802,-0.128843,-0.138543,0
BTC,-0.301184,-0.176797,-0.016435,0
ETH,-0.089452,-0.238902,-0.072992,0


In [170]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
clustered_df = pd.merge(crypto_df, pcs_df, left_index=True, right_index=True)
clustered_df.head(5)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,class
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.305859,-0.123153,-0.012508,0
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.297596,-0.127052,-0.007664,0
1337,X13,PoW/PoS,2.927942e+10,314159265359,0.303802,-0.128843,-0.138543,0
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.301184,-0.176797,-0.016435,0
ETH,Ethash,PoW,1.076842e+08,0,-0.089452,-0.238902,-0.072992,0


In [171]:
#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies.
clustered_df = pd.merge(clustered_df, coins_name, left_index=True, right_index=True)
clustered_df.head(5)

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,class,CoinName
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.305859,-0.123153,-0.012508,0,42 Coin
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.297596,-0.127052,-0.007664,0,404Coin
1337,X13,PoW/PoS,2.927942e+10,314159265359,0.303802,-0.128843,-0.138543,0,EliteCoin
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.301184,-0.176797,-0.016435,0,Bitcoin
ETH,Ethash,PoW,1.076842e+08,0,-0.089452,-0.238902,-0.072992,0,Ethereum


In [172]:
# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head(10)

(532, 9)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,class,CoinName
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.305859,-0.123153,-0.012508,0,42 Coin
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.297596,-0.127052,-0.007664,0,404Coin
1337,X13,PoW/PoS,2.927942e+10,314159265359,0.303802,-0.128843,-0.138543,0,EliteCoin
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.301184,-0.176797,-0.016435,0,Bitcoin
ETH,Ethash,PoW,1.076842e+08,0,-0.089452,-0.238902,-0.072992,0,Ethereum
LTC,Scrypt,PoW,6.303924e+07,84000000,-0.314537,-0.258179,-0.040234,0,Litecoin
DASH,X11,PoW/PoS,9.031294e+06,22000000,-0.218359,-0.028985,-0.010117,0,Dash
XMR,CryptoNight-V7,PoW,1.720114e+07,0,0.000950,-0.230373,-0.071575,0,Monero
ETC,Ethash,PoW,1.133597e+08,210000000,-0.259330,-0.233943,-0.067682,0,Ethereum Classic
ZEC,Equihash,PoW,7.383056e+06,21000000,-0.298687,-0.142293,-0.047664,0,ZCash


### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [173]:
# Creating a 3D-Scatter with the PCA data and the clusters
# 1. Create a 3D scatter plot. 
fig = px.scatter_3d(
    clustered_df,
    x="TotalCoinsMined",
    y="TotalCoinSupply",
    z="ProofType",
    color="class",    
    symbol="class",   
    hover_name="CoinName",
    hover_data=["Algorithm"],
    width=800,
)
fig.update_layout(legend=dict(x=0, y=1)) 
fig.show()

In [174]:
# Create a table with tradable cryptocurrencies.
clustered_df= clustered_df[['CoinName', 'Algorithm', 'ProofType', 'TotalCoinSupply', 'TotalCoinsMined', 'class']]
clustered_df.hvplot.table(width=800)

:Table   [CoinName,Algorithm,ProofType,TotalCoinSupply,TotalCoinsMined,class]

In [175]:
# Print the total number of tradable cryptocurrencies.
clustered_df.shape

(532, 6)

In [176]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.
minmaxscaler = MinMaxScaler().fit_transform(clustered_df[["TotalCoinSupply", "TotalCoinsMined"]])
print(minmaxscaler[0:5])

[[4.20000000e-11 0.00000000e+00]
 [5.32000000e-04 1.06585544e-03]
 [3.14159265e-01 2.95755135e-02]
 [2.10000000e-05 1.81084216e-05]
 [0.00000000e+00 1.08773140e-04]]


In [177]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
Clustered_df = pd.DataFrame(
    data=minmaxscaler, columns=["TotalCoinSupply", "TotalCoinsMined"], index=clustered_df.index)

# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
Clustered_df["CoinName"]=coins_name["CoinName"]

# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
# YOUR CODE HERE

Clustered_df["Class"] = model.labels_
Clustered_df.head(10)

,TotalCoinSupply,TotalCoinsMined,CoinName,Class
42,4.200000e-11,0.000000,42 Coin,0
404,5.320000e-04,0.001066,404Coin,0
1337,3.141593e-01,0.029576,EliteCoin,0
BTC,2.100000e-05,0.000018,Bitcoin,0
ETH,0.000000e+00,0.000109,Ethereum,0
LTC,8.400000e-05,0.000064,Litecoin,0
DASH,2.200000e-05,0.000009,Dash,0
XMR,0.000000e+00,0.000017,Monero,0
ETC,2.100000e-04,0.000115,Ethereum Classic,0
ZEC,2.100000e-05,0.000007,ZCash,0


In [178]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
clustered_df.hvplot.scatter(
x="TotalCoinsMined",
y="TotalCoinSupply",
    width=900, height=500,
hover_cols=["CoinName"],
by="class",
)

:NdOverlay   [class]
   :Scatter   [TotalCoinsMined]   (TotalCoinSupply,CoinName)